## Load notebooks from github

This cell will download notebooks and put them in the **examples** directory in two subdirectories:
* service - notebooks for use with the cuOpt service
* sdk - notebooks for use with the cuOpt Python SDK


In [ ]:
%%bash
mkdir -p examples/service
mkdir examples/sdk
git clone  -b branch-25.02 --single-branch https://github.com/NVIDIA/cuopt-examples
mv cuopt-examples/LICENSE.md cuopt-examples/README.md ./examples

# Service notebooks
for d in cuFOLIO_portfolio_optimization sample_lp_sever_notebooks routing_optimization_over_server; do
    mv cuopt-examples/$d ./examples/service/
done

# SDK notebooks
for d in last_mile_delivery intra-factory_transport; do
    mv cuopt-examples/$d ./examples/sdk/
done
rm -rf cuopt-examples

# Install Additional Libraries

The following cell installs some necessary libraries that may not be present on the system

In [ ]:
CUDA_VERSION = "12"
CUDA_SUFFIX = f"cu{CUDA_VERSION}"

In [ ]:
%%bash -s "$CUDA_VERSION" "$CUDA_SUFFIX"

# Configuration
CUDA_VERSION="$1"
CUDA_SUFFIX="$2"

# Function to check if a library exists
check_lib() {
    ldconfig -p | grep -q "$1"
    return $?
}

# Function to find library path
find_lib() {
    sudo find / -name "$1" 2>/dev/null | head -n 1
}

# Function to add path to ld.so.conf
add_to_ldconfig() {
    local lib_dir="$1"
    if [ -n "$lib_dir" ]; then
        echo "$lib_dir" | sudo tee -a /etc/ld.so.conf.d/cuda-${CUDA_SUFFIX}.conf >/dev/null
    fi
}

echo "Setting up CUDA ${CUDA_VERSION} libraries..."

# Install and setup CUDART if needed
if ! check_lib "libcudart.so.${CUDA_VERSION}"; then
    echo "Installing nvidia-cuda-runtime-${CUDA_SUFFIX}..."
    sudo pip3 install nvidia-cuda-runtime-${CUDA_SUFFIX}
    
    # Find CUDART library
    CUDART_PATH=$(find_lib "libcudart.so.${CUDA_VERSION}")
    if [ -z "$CUDART_PATH" ]; then
        echo "Error: Could not find libcudart.so.${CUDA_VERSION}"
        exit 1
    fi
    
    # Create symlink for libcudart.so
    CUDART_DIR=$(dirname "$CUDART_PATH")
    if [ ! -f "${CUDART_DIR}/libcudart.so" ]; then
        echo "Creating symlink for libcudart.so..."
        sudo ln -s "$CUDART_PATH" "${CUDART_DIR}/libcudart.so"
    fi
    
    # Add to LD_LIBRARY_PATH
    add_to_ldconfig "$CUDART_DIR"
else
    echo "CUDART library already installed"
fi

# Install and setup NVRTC if needed
if ! check_lib "libnvrtc.so.${CUDA_VERSION}"; then
    echo "Installing nvidia-cuda-nvrtc-${CUDA_SUFFIX}..."
    sudo pip3 install nvidia-cuda-nvrtc-${CUDA_SUFFIX}
    
    # Find NVRTC library
    NVRTC_PATH=$(find_lib "libnvrtc.so.${CUDA_VERSION}")
    if [ -z "$NVRTC_PATH" ]; then
        echo "Error: Could not find libnvrtc.so.${CUDA_VERSION}"
        exit 1
    fi
    
    # Add to LD_LIBRARY_PATH
    NVRTC_DIR=$(dirname "$NVRTC_PATH")
    add_to_ldconfig "$NVRTC_DIR"
else
    echo "NVRTC library already installed"
fi

In [ ]:
%%bash
echo "Updating ldconfig..."
sudo ldconfig

### COPYRIGHT and LICENSE

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
